El objetivo de este script es acceder a los experimentos guardados en comet y calcular distintos valores estadísticos

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
# Instalar comet en caso que sea necesario
!pip install comet_ml

In [3]:
import numpy as np
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd

# Indice

Voy a extraer las métricas de los experimentos asociados al *baseline*.


1.   *GBM* separando los datos por **sujeto**.
2.   *MLP* separando los datos por **sujeto**.
3.   *GBM* separando los datos por **repetición**.
3.   *MLP* separando los datos por **repetición**.



In [4]:
# Me conecto a Comet usando la misma key que uso para guardar los experimentos
api = API(api_key="ehXeElNypcj7Knar5zTmyjwSO")

Documentación de objetivo tipo API

https://www.comet.com/docs/v2/api-and-sdk/python-sdk/reference/API/#apiget_experiments

# *GBM* separando los datos por **sujeto**
Comet: baseline_gbm_sep_sub(i)_test

In [26]:
# Me fijo si estoy accediendo a los experimentos que quiero, y extraigo el set completo de sus metricas
patron = 'baseline_gbm_sep_sub[0-9]|10_test' # segmento de texto que debe tener cada experimento que quiera levantar

lista_exp = api.get_experiments(workspace = 'manuelmolinach99', project_name='tesis-experimentos', pattern=patron)
nombres = []
metricas = []
for i in range(len(lista_exp)):
# lista_exp[0].get_metrics()
  nombres.append(lista_exp[i].get_name())
  metricas.append(lista_exp[i].get_metrics(metric=None))
print(nombres)

# si los nombres son correctos, procedo a levantar las metricas de interes

['baseline_gbm_sep_sub1_test', 'baseline_gbm_sep_sub2_test', 'baseline_gbm_sep_sub3_test', 'baseline_gbm_sep_sub4_test', 'baseline_gbm_sep_sub5_test', 'baseline_gbm_sep_sub6_test', 'baseline_gbm_sep_sub7_test', 'baseline_gbm_sep_sub8_test', 'baseline_gbm_sep_sub9_test', 'baseline_gbm_sep_sub10_test']


In [ ]:
# hago una lista con las metricas que quiero extraer
nombres_metricas = ['accuracy', 'recall', 'precision', 'training_time']
accuracy_list = []  # 10 valores de accuracy, uno por experimento/sujeto
recall_list = []  # 10 valores de accuracy, uno por experimento/sujeto
precision_list = []  # 10 valores de accuracy, uno por experimento/sujeto
training_time_list = []  # 10 valores de accuracy, uno por experimento/sujeto
sujeto_list = list(np.arange(1, 11))

cant_experimentos = 10

for i in range(cant_experimentos):
  for dato in metricas[i]:
      # Verificar si el valor de 'metricName' es 'accuracy'
      if dato["metricName"] == "accuracy":
          # Si es así, guardar el valor correspondiente
          accuracy_list.append(dato["metricValue"])
      elif dato["metricName"] == "recall":
          # Si es así, guardar el valor correspondiente
          recall_list.append(dato["metricValue"])
      elif dato["metricName"] == "precision":
          # Si es así, guardar el valor correspondiente
          precision_list.append(dato["metricValue"])
      elif dato["metricName"] == "training_time":
          # Si es así, guardar el valor correspondiente
          training_time_list.append(dato["metricValue"])

# Crear un DataFrame con los datos
df = pd.DataFrame({
    'sujeto': sujeto_list,
    'accuracy': accuracy_list,
    'recall': recall_list,
    'precision': precision_list,
    'training_time': training_time_list
})

## Visualización de resultados

In [ ]:
# Imprimo dataFrame que contiene las metricas
df

,sujeto,accuracy,recall,precision,training_time
0,1,0.27869737315581145,0.27869737315581145,0.3012994747136013,133.18554544448853
1,2,0.3806186195337498,0.3806186195337498,0.40507196324268663,122.9550244808197
2,3,0.3686295696020169,0.3686295696020169,0.42056257932227964,122.84103488922119
3,4,0.3623021582733813,0.3623021582733813,0.4094796285828598,124.09736466407776
4,5,0.36556821681552204,0.36556821681552204,0.3820216012793683,123.37023448944092
5,6,0.23054545454545455,0.23054545454545455,0.24365059194765656,124.71125149726868
6,7,0.2956419316843345,0.2956419316843345,0.2967914975448593,121.90121006965637
7,8,0.32491968793024323,0.32491968793024323,0.3317811935824186,123.95904040336609
8,9,0.33389544688026984,0.33389544688026984,0.38829491572171315,122.61598944664001
9,10,0.23304894476874718,0.23304894476874718,0.23344892805828524,124.305011510849


In [ ]:
# Convierto las columnas a números
df = df.apply(pd.to_numeric, errors='coerce')

# Calculo las estadísticas sobre cada columna
media = df.mean()
mediana = df.median()
desviacion_estandar = df.std()
maximo = df.max()
minimo = df.min()
recuento = df.count()

# Creo un nuevo DataFrame con estas estadísticas
estadisticas_por_columna = pd.DataFrame({
    'Media': media,
    'Mediana': mediana,
    'Desviación Estándar': desviacion_estandar,
    'Máximo': maximo,
    'Mínimo': minimo,
    'Recuento': recuento
})

# Mostrar las estadísticas por columna
estadisticas_por_columna

,Media,Mediana,Desviación Estándar,Máximo,Mínimo,Recuento
sujeto,5.500000,5.500000,3.027650,10.000000,1.000000,10
accuracy,0.317387,0.329408,0.055694,0.380619,0.230545,10
recall,0.317387,0.329408,0.055694,0.380619,0.230545,10
precision,0.341240,0.356901,0.069685,0.420563,0.233449,10
training_time,124.394171,123.664637,3.207332,133.185545,121.901210,10


# *MLP* separando los datos por **sujeto**
Comet: baseline_mlp_sep_sub(i)_test

In [25]:
# Me fijo si estoy accediendo a los experimentos que quiero, y extraigo el set completo de sus metricas
patron = 'baseline_mlp_sep_sub[0-9]|10_test' # segmento de texto que debe tener cada experimento que quiera levantar

lista_exp = api.get_experiments(workspace = 'manuelmolinach99', project_name='tesis-experimentos', pattern=patron)
nombres = []
metricas = []
for i in range(len(lista_exp)):
# lista_exp[0].get_metrics()
  nombres.append(lista_exp[i].get_name())
  metricas.append(lista_exp[i].get_metrics(metric=None))
print(nombres)

# si los nombres son correctos, procedo a levantar las metricas de interes

['baseline_mlp_sep_sub1_test', 'baseline_mlp_sep_sub2_test', 'baseline_mlp_sep_sub3_test', 'baseline_mlp_sep_sub4_test', 'baseline_mlp_sep_sub5_test', 'baseline_mlp_sep_sub6_test', 'baseline_mlp_sep_sub7_test', 'baseline_mlp_sep_sub8_test', 'baseline_mlp_sep_sub9_test', 'baseline_mlp_sep_sub10_test']


In [ ]:
# hago una lista con las metricas que quiero extraer
nombres_metricas = ['accuracy', 'recall', 'precision', 'training_time']
accuracy_list = []  # 10 valores de accuracy, uno por experimento/sujeto
recall_list = []  # 10 valores de accuracy, uno por experimento/sujeto
precision_list = []  # 10 valores de accuracy, uno por experimento/sujeto
training_time_list = []  # 10 valores de accuracy, uno por experimento/sujeto
sujeto_list = list(np.arange(1, 11))

cant_experimentos = 10

for i in range(cant_experimentos):
  for dato in metricas[i]:
      # Verificar si el valor de 'metricName' es 'accuracy'
      if dato["metricName"] == "accuracy":
          # Si es así, guardar el valor correspondiente
          accuracy_list.append(dato["metricValue"])
      elif dato["metricName"] == "recall":
          # Si es así, guardar el valor correspondiente
          recall_list.append(dato["metricValue"])
      elif dato["metricName"] == "precision":
          # Si es así, guardar el valor correspondiente
          precision_list.append(dato["metricValue"])
      elif dato["metricName"] == "training_time":
          # Si es así, guardar el valor correspondiente
          training_time_list.append(dato["metricValue"])

# Crear un DataFrame con los datos
df = pd.DataFrame({
    'sujeto': sujeto_list,
    'accuracy': accuracy_list,
    'recall': recall_list,
    'precision': precision_list,
    'training_time': training_time_list
})

## Visualización de resultados

In [ ]:
df

,sujeto,accuracy,recall,precision,training_time
0,1,0.28085642317380355,0.28085642317380355,0.2995637382892417,803.7213940620422
1,2,0.3190614048453451,0.3190614048453451,0.34813911730358815,749.6828725337982
2,3,0.2593192868719611,0.2593192868719611,0.24921174817941377,802.804957151413
3,4,0.2814388489208633,0.2814388489208633,0.297793611373424,746.8992819786072
4,5,0.3478595626732368,0.3478595626732368,0.3599725165208878,743.1653938293457
5,6,0.1693090909090909,0.1693090909090909,0.18863914017259684,729.2504925727844
6,7,0.19964664310954064,0.19964664310954064,0.18107458939861262,802.8211557865143
7,8,0.2654122686247514,0.2654122686247514,0.2788538090999262,802.7690558433533
8,9,0.26598191016403494,0.26598191016403494,0.26642589907952163,802.8755395412445
9,10,0.18155964675946715,0.18155964675946715,0.1740001041962186,753.3965444564819


In [ ]:
# Convierto las columnas a números
df = df.apply(pd.to_numeric, errors='coerce')

# Calculo las estadísticas sobre cada columna
media = df.mean()
mediana = df.median()
desviacion_estandar = df.std()
maximo = df.max()
minimo = df.min()
recuento = df.count()

# Creo un nuevo DataFrame con estas estadísticas
estadisticas_por_columna = pd.DataFrame({
    'Media': media,
    'Mediana': mediana,
    'Desviación Estándar': desviacion_estandar,
    'Máximo': maximo,
    'Mínimo': minimo,
    'Recuento': recuento
})

# Mostrar las estadísticas por columna
estadisticas_por_columna

,Media,Mediana,Desviación Estándar,Máximo,Mínimo,Recuento
sujeto,5.500000,5.500000,3.027650,10.000000,1.000000,10
accuracy,0.257045,0.265697,0.057832,0.347860,0.169309,10
recall,0.257045,0.265697,0.057832,0.347860,0.169309,10
precision,0.264367,0.272640,0.066497,0.359973,0.174000,10
training_time,773.738669,778.082800,31.460882,803.721394,729.250493,10


# *GBM* separando los datos por **repetición**

Comet: baseline_gbm_sep_rep(i)_test

In [24]:
# Me fijo si estoy accediendo a los experimentos que quiero, y extraigo el set completo de sus metricas
patron = 'baseline_gbm_sep_rep[0-9]|10_test' # segmento de texto que debe tener cada experimento que quiera levantar

lista_exp = api.get_experiments(workspace = 'manuelmolinach99', project_name='tesis-experimentos', pattern=patron)
nombres = []
metricas = []
for i in range(len(lista_exp)):
# lista_exp[0].get_metrics()
  nombres.append(lista_exp[i].get_name())
  metricas.append(lista_exp[i].get_metrics(metric=None))
print(nombres)

# si los nombres son correctos, procedo a levantar las metricas de interes

['baseline_gbm_sep_rep1_test', 'baseline_gbm_sep_rep2_test', 'baseline_gbm_sep_rep3_test', 'baseline_gbm_sep_rep4_test', 'baseline_gbm_sep_rep5_test', 'baseline_gbm_sep_rep6_test', 'baseline_gbm_sep_rep7_test', 'baseline_gbm_sep_rep8_test', 'baseline_gbm_sep_rep9_test', 'baseline_gbm_sep_rep10_test']


In [7]:
# hago una lista con las metricas que quiero extraer
nombres_metricas = ['accuracy', 'recall', 'precision', 'training_time']
accuracy_list = []  # 10 valores de accuracy, uno por experimento/sujeto
recall_list = []  # 10 valores de accuracy, uno por experimento/sujeto
precision_list = []  # 10 valores de accuracy, uno por experimento/sujeto
training_time_list = []  # 10 valores de accuracy, uno por experimento/sujeto
sujeto_list = list(np.arange(1, 11))

cant_experimentos = 10

for i in range(cant_experimentos):
  for dato in metricas[i]:
      # Verificar si el valor de 'metricName' es 'accuracy'
      if dato["metricName"] == "accuracy":
          # Si es así, guardar el valor correspondiente
          accuracy_list.append(dato["metricValue"])
      elif dato["metricName"] == "recall":
          # Si es así, guardar el valor correspondiente
          recall_list.append(dato["metricValue"])
      elif dato["metricName"] == "precision":
          # Si es así, guardar el valor correspondiente
          precision_list.append(dato["metricValue"])
      elif dato["metricName"] == "training_time":
          # Si es así, guardar el valor correspondiente
          training_time_list.append(dato["metricValue"])

# Crear un DataFrame con los datos
df = pd.DataFrame({
    'sujeto': sujeto_list,
    'accuracy': accuracy_list,
    'recall': recall_list,
    'precision': precision_list,
    'training_time': training_time_list
})

## Visualización de resultados

In [8]:
# Imprimo dataFrame que contiene las metricas
df

,sujeto,accuracy,recall,precision,training_time
0,1,0.5750275547157928,0.5750275547157928,0.5998559180147791,167.77149891853333
1,2,0.6059462010382256,0.6059462010382256,0.6287950447537873,155.18143963813782
2,3,0.5270681265206812,0.5270681265206812,0.5428415687424583,148.98502731323242
3,4,0.5668819914920434,0.5668819914920434,0.5865032076675991,149.52908301353455
4,5,0.585029752583777,0.585029752583777,0.5988318839418489,146.7278962135315
5,6,0.5824143070044709,0.5824143070044709,0.6035404284882233,145.85118699073792
6,7,0.5549177784672665,0.5549177784672665,0.5642066608942902,147.45593810081482
7,8,0.5986078886310905,0.5986078886310905,0.6129790417329818,141.56677508354187
8,9,0.5647374231193818,0.5647374231193818,0.5861373048699524,145.82603287696838
9,10,0.5726286848938446,0.5726286848938446,0.5921655102086516,143.90957951545715


In [9]:
# Convierto las columnas a números
df = df.apply(pd.to_numeric, errors='coerce')

# Calculo las estadísticas sobre cada columna
media = df.mean()
mediana = df.median()
desviacion_estandar = df.std()
maximo = df.max()
minimo = df.min()
recuento = df.count()

# Creo un nuevo DataFrame con estas estadísticas
estadisticas_por_columna = pd.DataFrame({
    'Media': media,
    'Mediana': mediana,
    'Desviación Estándar': desviacion_estandar,
    'Máximo': maximo,
    'Mínimo': minimo,
    'Recuento': recuento
})

# Mostrar las estadísticas por columna
estadisticas_por_columna

,Media,Mediana,Desviación Estándar,Máximo,Mínimo,Recuento
sujeto,5.500000,5.500000,3.027650,10.000000,1.000000,10
accuracy,0.573326,0.573828,0.022411,0.605946,0.527068,10
recall,0.573326,0.573828,0.022411,0.605946,0.527068,10
precision,0.591586,0.595499,0.024238,0.628795,0.542842,10
training_time,149.280446,147.091917,7.444416,167.771499,141.566775,10


# *MLP* separando los datos por **repetición**

Comet: baseline_mlp_sep_rep(i)_test

In [29]:
# Me fijo si estoy accediendo a los experimentos que quiero, y extraigo el set completo de sus metricas
patron = 'baseline_mlp_sep_rep[0-9]|10_test' # segmento de texto que debe tener cada experimento que quiera levantar

lista_exp = api.get_experiments(workspace = 'manuelmolinach99', project_name='tesis-experimentos', pattern=patron)
nombres = []
metricas = []
for i in range(len(lista_exp)):
  nombres.append(lista_exp[i].get_name())
  metricas.append(lista_exp[i].get_metrics(metric=None))
print(nombres)

['baseline_mlp_sep_rep1_test', 'baseline_mlp_sep_rep2_test', 'baseline_mlp_sep_rep3_test', 'baseline_mlp_sep_rep4_test', 'baseline_mlp_sep_rep5_test', 'baseline_mlp_sep_rep6_test', 'baseline_mlp_sep_rep7_test', 'baseline_mlp_sep_rep8_test', 'baseline_mlp_sep_rep9_test', 'baseline_mlp_sep_rep10_test']


In [30]:
# hago una lista con las metricas que quiero extraer
nombres_metricas = ['accuracy', 'recall', 'precision', 'training_time']
accuracy_list = []  # 10 valores de accuracy, uno por experimento/sujeto
recall_list = []  # 10 valores de accuracy, uno por experimento/sujeto
precision_list = []  # 10 valores de accuracy, uno por experimento/sujeto
training_time_list = []  # 10 valores de accuracy, uno por experimento/sujeto
sujeto_list = list(np.arange(1, 11))

cant_experimentos = 10

for i in range(cant_experimentos):
  for dato in metricas[i]:
      # Verificar si el valor de 'metricName' es 'accuracy'
      if dato["metricName"] == "accuracy":
          # Si es así, guardar el valor correspondiente
          accuracy_list.append(dato["metricValue"])
      elif dato["metricName"] == "recall":
          # Si es así, guardar el valor correspondiente
          recall_list.append(dato["metricValue"])
      elif dato["metricName"] == "precision":
          # Si es así, guardar el valor correspondiente
          precision_list.append(dato["metricValue"])
      elif dato["metricName"] == "training_time":
          # Si es así, guardar el valor correspondiente
          training_time_list.append(dato["metricValue"])

# Crear un DataFrame con los datos
df = pd.DataFrame({
    'sujeto': sujeto_list,
    'accuracy': accuracy_list,
    'recall': recall_list,
    'precision': precision_list,
    'training_time': training_time_list
})

## Visualización de resultados

In [31]:
# Imprimo dataFrame que contiene las metricas
df

,sujeto,accuracy,recall,precision,training_time
0,1,0.5805384978743505,0.5805384978743505,0.5874255189795818,1104.014428138733
1,2,0.6168003775365739,0.6168003775365739,0.6289619833203526,1103.1834018230438
2,3,0.5492700729927007,0.5492700729927007,0.5559165109020104,1103.1091828346252
3,4,0.5823223570190641,0.5823223570190641,0.5967305560728222,1103.3988399505615
4,5,0.599279674287504,0.599279674287504,0.6162853762006795,1103.354813337326
5,6,0.6034277198211624,0.6034277198211624,0.6197466607785656,1077.5492677688599
6,7,0.5798945082221533,0.5798945082221533,0.5864938715267944,1102.9509978294373
7,8,0.6157772621809745,0.6157772621809745,0.6214137134824782,1094.0967721939087
8,9,0.5934395205803501,0.5934395205803501,0.5950251643716893,1163.1978549957275
9,10,0.6068428287765388,0.6068428287765388,0.6175858574182083,1162.9470601081848


In [32]:
# Convierto las columnas a números
df = df.apply(pd.to_numeric, errors='coerce')

# Calculo las estadísticas sobre cada columna
media = df.mean()
mediana = df.median()
desviacion_estandar = df.std()
maximo = df.max()
minimo = df.min()
recuento = df.count()

# Creo un nuevo DataFrame con estas estadísticas
estadisticas_por_columna = pd.DataFrame({
    'Media': media,
    'Mediana': mediana,
    'Desviación Estándar': desviacion_estandar,
    'Máximo': maximo,
    'Mínimo': minimo,
    'Recuento': recuento
})

# Mostrar las estadísticas por columna
estadisticas_por_columna

,Media,Mediana,Desviación Estándar,Máximo,Mínimo,Recuento
sujeto,5.500000,5.500000,3.027650,10.000000,1.000000,10
accuracy,0.592759,0.596360,0.020489,0.616800,0.549270,10
recall,0.592759,0.596360,0.020489,0.616800,0.549270,10
precision,0.602559,0.606508,0.022399,0.628962,0.555917,10
training_time,1111.780262,1103.269108,28.234870,1163.197855,1077.549268,10
